In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            for s in ['main', 'master']:
                uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
                x = requests.get(uri).text
                if x.status == 200:
                    break
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            for s in ['/raw/main/', '/raw/master/']:
                uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:]) + '.py'
                x = requests.get(uri).text
                if x.status == 200:
                    break
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()


CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed


In [2]:
from k12libs.utils.nb_easy import k12ai_start_html, W3URL
from urllib.parse import urlencode

## API (/k12ai/framework/execute)

In [3]:
SSS = 'http://116.85.5.40:8119'
EXE = '/k12ai/framework/execute'
EXE_URL = f'{SSS}{EXE}'
EXE_URL

'http://116.85.5.40:8119/k12ai/framework/execute'

### 调试输出(内部使用)

In [4]:
POP = '/k12ai/private/popmsg'
POP_URL = f'{SSS}{POP}'
POP_URL

'http://116.85.5.40:8119/k12ai/private/popmsg'

## 输入参数

```json
{
    'appId': 'k12pyr.16601548608.123456',  // [M] 用来异步redis传递数据的Key前缀, 用户日志:`$appId.runlog`, 代码错误:`$appId.error`
    'token': '916886',                     // [O] 随机token, 供业务自己使用, 只负责透传 
    'op': 'runcode.start',                 // [M] 操作类型(不可修改)
    'user': '16601548608',                 // [M] 用户ID(学生唯一码)
    'service_name': 'k12pyr',              // [M] 服务类型(不可修改)
    'service_uuid': '123456',              // [M] 服务编号(需要保证不同场景该值不同)
    'service_params': {                    // [M] 服务参数, 'code'参数是代码框里的字符串
        'code': "import os\nimport time\nfor i in range(10):\n    print('i = %d' % i)\n"
    }
}
```

### 测试

### $appId.error [见:统一错误码](http://116.85.5.40:8118/notebooks/docs/k12ai_errorcode.ipynb#统一错误码)

- A.无错误时(expand内容为status)

```json
{
    'version': '0.1.0',
    'server': 'localhost:8119',
    'type': 'error',                          // 消息类型
    'appId': 'k12pyr.16601548608.123456',     // redis key(前缀)
    'token': '608997',
    'user': '16601548608',
    'op': 'runcode.start',
    'service_name': 'k12pyr',
    'service_uuid': '123456',
    'timestamp': 1628762229417,               // 日期时间戳
    'datetime': '20210812175709',             // 当前日期
    'data': {                                 // 消息负载
        'code': 100003,                       // 错误码
        'message': {'en': 'program normal exit', 'cn': '程序正常结束'},  // 错误码对应的消息内容
        'expand': {'status': 'finished'}      // 错误码对应的扩展消息 (当没错时:主要是状态信息, 当有错误时: 主要是堆栈信息)
    }
}
```

- B.有错误时(expand内容为errinfo)

```json
{
    "version": "0.1.0",
    "server": "localhost:8119",
    "type": "error",                                     // 消息类型
    "appId": "k12pyr.16601548608.123456",
    "token": "930538",
    "user": "16601548608",
    "op": "runcode.start",
    "service_name": "k12pyr",
    "service_uuid": "123456",
    "timestamp": 1628764147761,
    "datetime": "20210812182907",
    "data": {
        "code": 100910,
        "message": {
            "en": "import module can not found",
            "cn": "常见错误: 引入的模块不存在"
        },
        "expand": {
            "errinfo": {                                 // 程序错误信息
                "err_type": "ModuleNotFoundError",       // 错误类型
                "err_text": "No module named 'xxtime'",  // 错误类型消息内容
                "trackback": [                           // 错误信息的堆栈
                    {
                        "filename": "pyrcode",           // 错误发生的文件 (针对代码块执行, 默认都是pyrcode)
                        "linenum": 2                     // 错误发生在第几行数
                    }
                ]
            }
        }
    }
}
```

### $appId.runlog (用户代码操作输出)

- A. 正常用户的print

```json
{
    "version": "0.1.0",
    "server": "localhost:8119",
    "type": "runlog",                           // 消息类型
    "appId": "k12pyr.16601548608.123456",
    "token": "649147",
    "user": "16601548608",
    "op": "runcode.start",
    "service_name": "k12pyr",
    "service_uuid": "123456",
    "timestamp": 1628764719553,
    "datetime": "20210812183839",
    "data": {                                   // 消息负载
        "log": "i = 9\n"                        // print的消息内容
    }
}
```

- B. 带有图片操作show

```json
{
    "version": "0.1.0",
    "server": "localhost:8119",
    "type": "runlog",                          // 消息类型
    "appId": "k12pyr.16601548608.123456",
    "token": "890443",
    "user": "16601548608",
    "op": "runcode.start",
    "service_name": "k12pyr",
    "service_uuid": "123456",
    "timestamp": 1628765202398,
    "datetime": "20210812184642",
    "data": {                                  // 消息负载 (imshow base64 string)
        "imshow": "iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAAB30lEQVR4nG2SvWtTYRTGf+9X7leTNkpACLQ6iYq2iEgHB6mCqKAUUeifoK7i4tBRKgouDo51cRP8WBykUFsQA4LuVVQogiXNNc0Hvck9DsmNXPUZz/O+v/d5zzngYyMgAgIMDu3QHgONAYwV0GVVGJY0OACsxiiyk9G4VwJnUQAEUPQHABMAYFB6hLAWCFEKfMc+CME4AIWlR9iGszOlyvkp3t1/JqBML3s9DNn/8IvIpkhHPpxGDfhYUpd0072LC97Gp9ettHxr5tJaL8X0B67C6vIPqV/QBJQ2ZJ4QhpG0BszjhryZP0ORWC56HnuGWLcL9J8cmp2dq3+UidLXg/H1qLLwDQAHSikXHb/3VkRaaXdb2uu3yWQtHlDh8ve0K9K8M1WMsrgojQONOtHqyI7UGB92nAIeCk2If/T5L9mS7W7NMMGwk0WUNhBUFvvy6lw1lhWMsuSk1kRuRvBSXvgQ5Dx7rSPLvmP6Z32d4cRGOvI5lknP52RTah7G5MzJrVQO4OunSeOqsuRNVqR7jOqjRN6PmKNU3ib9U3PVG7AaJEk2z2wjrki820h3kuXD4P31E6JVSUXad5XWeAPin8yFwtJ0rfkg5j8agwiKuVq2hPQIBNUt/HNrIAWlfOU3AZKW2kL7nCQAAAAASUVORK5CYII="
    }
}
```

In [30]:
code_100905 = '''
from PIL import Image
image = Image.open('/noexist.png') # error
image.show()
'''

code_100910 = '''
import oss # error
'''

code_100911 = '''
a = [1, 2, 3]
print(a[10]) # error
'''

code_100912 = '''
a = b # error
'''

code_100913 = '''
a = {'name': 'li'}
print(a['phone']) # error
'''

code_100914 = '''
a = {'name': 'li'}
print(a.name) # error
'''

code_100915 = '''
if 1 > 2 {         # error
    print('1 > 2')
}
'''

code_100916 = '''
a = 10
 b = 30        # error
print(a, b)
'''

code_100917 = '''
a = [1, 2, 3]
print(a['1']) # error
'''

code_100918 = '''
a = 10 + 'abc'   # error
'''

code_100930 = '''
# 错误
a = 1 / 0  # 错误
'''

code_100999 = '''

'''

params = {'default': code_100930}
k12ai_start_html(f'{W3URL}/codemirror.html?{urlencode(params)}', width='100%', height='1400px')